In [1]:
import numpy as np
from keras.applications import VGG16, MobileNet
from keras.preprocessing.image import ImageDataGenerator

# Define image dimensions
img_width, img_height = 224, 224

# Load VGG16 pre-trained model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Load MobileNet pre-trained model
mobilenet_model = MobileNet(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Define data generators
train_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Set your training and testing directories
train_dir = "C:\\Users\\tumma\\Desktop\\skincancerdetection\\train"
test_dir = "C:\\Users\\tumma\\Desktop\\skincancerdetection\\test"

# Define batch size
batch_size = 32

# Generate training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,  # This means our generator will only yield batches of data, no labels
    shuffle=False)  # Important to keep the same order as our labels

# Generate testing data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

# Extract features using VGG16
X_train_features = vgg_model.predict(train_generator, verbose=1)
X_test_features = vgg_model.predict(test_generator, verbose=1)

# Extract features using MobileNet
mobilenet_X_train_features = mobilenet_model.predict(train_generator, verbose=1)
mobilenet_X_test_features = mobilenet_model.predict(test_generator, verbose=1)





17225924/17225924 [==============================] - 3s 0us/step
Found 11879 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
63/63 [==============================] - 26s 403ms/step


In [22]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D, Concatenate

# Apply Global Average Pooling to the feature maps
vgg_pooled_features = GlobalAveragePooling2D()(X_train_features)
mobilenet_pooled_features = GlobalAveragePooling2D()(mobilenet_X_train_features)

# Concatenate the pooled feature maps
concatenated_X_train_features = Concatenate(axis=1)([vgg_pooled_features, mobilenet_pooled_features])

# Apply the same operations to the test data
vgg_pooled_features_test = GlobalAveragePooling2D()(X_test_features)
mobilenet_pooled_features_test = GlobalAveragePooling2D()(mobilenet_X_test_features)
concatenated_X_test_features = Concatenate(axis=1)([vgg_pooled_features_test, mobilenet_pooled_features_test])
y_train = train_generator.classes
# Get the actual class labels for testing data
y_test = test_generator.classes


# Define the model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(concatenated_X_train_features.shape[1],)),  
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()
from keras.callbacks import EarlyStopping
import warnings 
warnings.filterwarnings('ignore')
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
# Train the model
history = model.fit(concatenated_X_train_features, y_train, epochs=5, batch_size=32, validation_data=(concatenated_X_test_features, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(concatenated_X_test_features, y_test)
print('Test accuracy:', test_acc)

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_75 (Dense)            (None, 128)               196736    
                                                                 
 dropout_57 (Dropout)        (None, 128)               0         
                                                                 
 dense_76 (Dense)            (None, 64)                8256      
                                                                 
 dropout_58 (Dropout)        (None, 64)                0         
                                                                 
 dense_77 (Dense)            (None, 32)                2080      
                                                                 
 dropout_59 (Dropout)        (None, 32)                0         
                                                                 
 dense_78 (Dense)            (None, 2)               

In [29]:

from keras.preprocessing import image
import numpy as np

# Define the function to preprocess the image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match batch size
    img_array /= 255.  # Normalize pixel values
    return img_array

# Define the function to predict the class of the image
def predict_image_class(img_path):
    preprocessed_img = preprocess_image(img_path)
    # Extract features using both VGG16 and MobileNet
    vgg_features = vgg_model.predict(preprocessed_img)
    mobilenet_features = mobilenet_model.predict(preprocessed_img)
    # Apply Global Average Pooling to the feature maps
    vgg_pooled_features = np.mean(vgg_features, axis=(1, 2))
    mobilenet_pooled_features = np.mean(mobilenet_features, axis=(1, 2))
    # Concatenate the pooled feature maps
    concatenated_features = np.concatenate([vgg_pooled_features, mobilenet_pooled_features], axis=1)
    predictions = model.predict(concatenated_features)
    # Interpret predictions
    if predictions[0][0] > predictions[0][1]:
        return "Benign"
    else:
        return "Malignant"

# Test the function on an example image
test_image_path = "C:\\Users\\tumma\\Desktop\\skincancerdetection\\test\\Benign\\6310.jpg"  # Provide the path to your test image
predicted_class = predict_image_class(test_image_path)
print("Predicted class:", predicted_class)


1/1 [==============================] - 1s 729ms/step
Predicted class: Benign
